# Biopython 4: Exploring the Cookbook

The Biopython Tutorial has a [Cookbook chapter](https://biopython.org/DIST/docs/tutorial/Tutorial.html#sec429), as well as a [user-contributed Cookbook](https://biopython.org/wiki/Category:Cookbook) in the documentation. We will now go through some useful "recipes" which may make your life a bit easier. We will use what we have already covered in the past workshops and check out some useful code examples. 

## Example 1: 22.1.1 Filtering a Sequence File

Often you’ll have a large file with many sequences in it (e.g. FASTA file or genes, or a FASTQ or SFF file of reads), a separate shorter list of the IDs for a subset of sequences of interest, and want to make a new sequence file for this subset.

Let’s say the list of IDs is in a simple text file, as the first word on each line. This could be a tabular file where the first column is the ID. Try something like this:

Note that we use a Python set rather than a list, this makes testing membership faster. A set is a python object used for storing multiple elements (similar to a list, tuple, or dictionary). The elements of a set are unique (meaning you cannot add an element to a set if the set already contains that element). You cannot index or slice a set because the emelents are not ordered. However, it supports mathematical operations such as unions, differences, and intersections of sets.

For a large file we may not need to make SeqRecord objects (it's faster to deal with strings). For FASTA files we used SimpleFastaParser - now we can use FastqGeneralIterator to achieve the same effect with a FASTQ file.

In [ ]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator

input_file = "big_file.fastq"
id_file = "short_list.txt"
output_file = "short_list.fastq"

with open(id_file) as id_handle:
    # Taking first word on each line as an identifier
    wanted = set(line.rstrip("\n").split(None, 1)[0] for line in id_handle)
print("Found %i unique identifiers in %s" % (len(wanted), id_file))

with open(input_file) as in_handle:
    with open(output_file, "w") as out_handle:
        for title, seq, qual in FastqGeneralIterator(in_handle):
            # The ID is the first word in the title line (after the @ sign):
            if title.split(None, 1)[0] in wanted:
                # This produces a standard 4-line FASTQ entry:
                out_handle.write("@%s\n%s\n+\n%s\n" % (title, seq, qual))
                count += 1
print("Saved %i records from %s to %s" % (count, input_file, output_file))
if count < len(wanted):
    print("Warning %i IDs not found in %s" % (len(wanted) - count, input_file))

## Example 2: 22.1.6 Quality filtering for FASTQ files

The FASTQ file format was introduced at Sanger and is now widely used for holding nucleotide sequencing reads together with their quality scores. FASTQ files (and the related QUAL files) are an excellent example of per-letter-annotation, because for each nucleotide in the sequence there is an associated quality score. Any per-letter-annotation is held in a SeqRecord in the letter_annotations dictionary as a list, tuple or string (with the same number of elements as the sequence length).

One common task is taking a large set of sequencing reads and filtering them (or cropping them) based on their quality scores. The following example is very simplistic, but should illustrate the basics of working with quality data in a SeqRecord object. All we are going to do here is read in a file of FASTQ data, and filter it to pick out only those records whose PHRED quality scores are all above some threshold (here 20).

For this example we’ll use some real data downloaded from the ENA sequence read archive, ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR020/SRR020192/SRR020192.fastq.gz (2MB) which unzips to a 19MB file SRR020192.fastq. This is some Roche 454 GS FLX single end data from virus infected California sea lions (see https://www.ebi.ac.uk/ena/data/view/SRS004476 for details).

First, let’s count the reads:

In [16]:
from Bio import SeqIO
import gzip

count = 0
with gzip.open("sample-data/SRR020192.fastq.gz", "rt") as handle:
    for rec in SeqIO.parse(handle, "fastq"):
        count += 1

print("%i reads" % count)

41892 reads


Now let’s do a simple filtering for a minimum PHRED quality of 20:

In [20]:
from Bio import SeqIO
with gzip.open("sample-data/SRR020192.fastq.gz", "rt") as handle:
    good_reads = (
        rec
        for rec in SeqIO.parse(handle, "fastq")
        if min(rec.letter_annotations["phred_quality"]) >= 20
    )
    count = SeqIO.write(good_reads, "sample-data/good_quality.fastq", "fastq")
print("Saved %i reads" % count)

Saved 39961 reads


## Example 3: Retrieving nonmatchine BLAST queries (from the user-contributed cookbook)

The XML output of NCBI’s stand alone BLAST programs does not include information on query sequences that have ‘no hits’ in the target database. Sometimes you want to know which sequences don’t have match a database and further analyse/annotate them accordingly. There are a number of different ways to do this, one is to use SeqIO’s method .index() to turn the query file into a dictionary then parse the results file to get the sequences that did match the dictionary. You can then use Python’s set() arithmetic to make a list of sequences that are in the query file and not the results which can be used as keys to retrieve the complete SeqRecord for each of the “no hit” queries. Got it? Well, perhaps it’s easier to just do it:

Let’s presume you set up a BLAST run with the sequences in a file called queries.fasta searched against a database, with the results saved to BLAST_RESULTS.xml

In [ ]:
from Bio import SeqIO
from Bio.Blast import NCBIXML

# Build an index, but we don't need to parse the record
q_dict = SeqIO.index("queries.fasta", "fasta")

hits = []
for record in NCBIXML.parse(open("BLAST_RESULTS.xml")):
    # As of BLAST 2.2.19 the xml output for multiple-query blast searches
    # skips queries with no hits so we could just append the ID of every blast
    # record to our 'hit list'. It's possible that the NCBI will change this
    # behaviour in the future so let's make the appending conditional on there
    # being some hits (ie, a non-empty alignments list) recorded in the blast
    # record

    if record.alignments:
        # The blast record's 'query' contains the sequences description as a
        # string. We used the ID as the key in our dictionary so we'll need to
        # split the string and get the first field to remove the right entries
        hits.append(record.query.split()[0])

misses = set(q_dict.keys()) - set(hits)
orphan_records = [q_dict[name] for name in misses]